In [3]:
# Import packages
import spotipy
import spotipy.util as util
import pickle
import sys
import requests
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
from multiprocessing import Pool
from fuzzywuzzy import fuzz
import string
import json
import unidecode
import sys
!{sys.executable} -m pip install pytrends
import time
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
from pytrends.request import TrendReq
from random import randint

In [4]:
# Read song data frame from pkl file
with open('song_df.pkl', 'rb') as f:
        song_df = pickle.load(f)

In [5]:
# Examine the dataframe
song_df.head()

,artist,title,isnew,rank,date,spotify_id,spotify_explicit,spotify_duration_ms,spotify_disc_number,spotify_track_number,...,spotify_acousticness,spotify_instrumentalness,spotify_liveness,spotify_valence,spotify_tempo,spotify_time_signature,spotify_album_id,spotify_album_release_date,spotify_album_release_date_precision,album_label
0,Lewis Capaldi,Someone You Loved,False,1,2019-11-16,7qEHsqek33rTcFNT9PFqLf,False,182160.0,1.0,4.0,...,0.751,0.00000,0.1050,0.4460,109.891,4.0,5658aM19fA3JVwTK6eQX70,2019-05-17,day,Vertigo Berlin
1,Post Malone,Circles,False,2,2019-11-16,21jGcNKet2qwijlDFuPiPb,False,215280.0,1.0,6.0,...,0.192,0.00244,0.0863,0.5530,120.042,4.0,4g1ZRSobMefqF6nelkgibi,2019-09-06,day,Republic Records
2,Shawn Mendes & Camila Cabello,Senorita,False,3,2019-11-16,0TK2YIli7K1leLovkQiNik,False,190960.0,1.0,1.0,...,0.037,0.00000,0.0945,0.7500,116.947,4.0,2ZaX1FdZCwchXl1QZiD4O4,2019-06-21,day,Island Records
3,Lizzo,Truth Hurts,False,4,2019-11-16,5qmq61DAAOUaW8AUo8xKhh,True,173325.0,1.0,13.0,...,0.110,0.00000,0.1230,0.4120,158.087,4.0,74gSdSHe71q7urGWMMn3qB,2019-05-03,day,Nice Life/Atlantic
4,Selena Gomez,Lose You To Love Me,False,5,2019-11-16,1HfMVBKM75vxSfsQ5VefZ5,False,206458.0,1.0,1.0,...,0.576,0.00000,0.2100,0.0916,101.993,4.0,3tBkjgxDqAwss76O1YHsSY,2019-10-23,day,Interscope Records


In [6]:
# Drop lines containing N/A
song_dropna = song_df.dropna().copy()
song_dropna.head()

,artist,title,isnew,rank,date,spotify_id,spotify_explicit,spotify_duration_ms,spotify_disc_number,spotify_track_number,...,spotify_acousticness,spotify_instrumentalness,spotify_liveness,spotify_valence,spotify_tempo,spotify_time_signature,spotify_album_id,spotify_album_release_date,spotify_album_release_date_precision,album_label
0,Lewis Capaldi,Someone You Loved,False,1,2019-11-16,7qEHsqek33rTcFNT9PFqLf,False,182160.0,1.0,4.0,...,0.751,0.00000,0.1050,0.4460,109.891,4.0,5658aM19fA3JVwTK6eQX70,2019-05-17,day,Vertigo Berlin
1,Post Malone,Circles,False,2,2019-11-16,21jGcNKet2qwijlDFuPiPb,False,215280.0,1.0,6.0,...,0.192,0.00244,0.0863,0.5530,120.042,4.0,4g1ZRSobMefqF6nelkgibi,2019-09-06,day,Republic Records
2,Shawn Mendes & Camila Cabello,Senorita,False,3,2019-11-16,0TK2YIli7K1leLovkQiNik,False,190960.0,1.0,1.0,...,0.037,0.00000,0.0945,0.7500,116.947,4.0,2ZaX1FdZCwchXl1QZiD4O4,2019-06-21,day,Island Records
3,Lizzo,Truth Hurts,False,4,2019-11-16,5qmq61DAAOUaW8AUo8xKhh,True,173325.0,1.0,13.0,...,0.110,0.00000,0.1230,0.4120,158.087,4.0,74gSdSHe71q7urGWMMn3qB,2019-05-03,day,Nice Life/Atlantic
4,Selena Gomez,Lose You To Love Me,False,5,2019-11-16,1HfMVBKM75vxSfsQ5VefZ5,False,206458.0,1.0,1.0,...,0.576,0.00000,0.2100,0.0916,101.993,4.0,3tBkjgxDqAwss76O1YHsSY,2019-10-23,day,Interscope Records


In [14]:
#this function returns the mean trend of the past n months of a song in song_df 
def google_trend(song_df, song, n):
    
    #set date
    trend_date = str(song_df.iloc[song]["spotify_album_release_date"])
    
    print(trend_date)
    if trend_date:
        if len(trend_date) > 5:
            dt = datetime.strptime(trend_date, '%Y-%m-%d')
        elif trend_date =='nan':
            dt = datetime.strptime('2000-1-1', '%Y-%m-%d')
        else:
            dt = datetime.strptime(trend_date, '%Y')
    
    print(dt)
    
    dt_start = dt + relativedelta(months=-n)
   
    trend_artist = song_df.iloc[song]["artist"].replace(',', '')
    
    if len(trend_artist) < 100:
        kw_list = [trend_artist]
    else:
        splitted = trend_artist.split()
        first = splitted[0]
        second = splitted[1]
        kw_list = [first + second]
    
    print(kw_list)
    
    #set dataframe, the earliset is 1/1/2004
    earliest_date = '2004-1-1'
    google_earliest = datetime.strptime(earliest_date, '%Y-%m-%d')
    if dt_start >= google_earliest:
        pytrends = TrendReq(hl='en-US', tz=360)
        time_period = dt_start.strftime("%Y") + '-' + dt_start.strftime("%m") + '-' + dt_start.strftime("%d") + ' ' + dt.strftime("%Y") + '-' + dt.strftime("%m") + '-' + dt.strftime("%d")
        
        print(time_period)
        pytrends.build_payload(kw_list, cat=0, timeframe= time_period, geo='', gprop='')
        trend = pytrends.interest_over_time()
        if not trend.empty:
            trend_mean=np.mean(trend.iloc[:,0])
            return trend_mean
        else:
            return 0
    elif dt_start < google_earliest and dt > google_earliest:
        pytrends = TrendReq(hl='en-US', tz=360)
        time_period = google_earliest.strftime("%Y") + '-' + google_earliest.strftime("%m") + '-' + google_earliest.strftime("%d") + ' ' + dt.strftime("%Y") + '-' + dt.strftime("%m") + '-' + dt.strftime("%d")
        pytrends.build_payload(kw_list, cat=0, timeframe= time_period, geo='', gprop='')
        trend = pytrends.interest_over_time()
        
        if not trend.empty:
            trend_mean=np.mean(trend.iloc[:,0])
            return trend_mean
        else:
            return 0
    elif dt <= google_earliest:
        trend_mean = -1
        return trend_mean

#test    
print(google_trend(song_df, 13, 3))

2019-10-04
2019-10-04 00:00:00
['Travis Scott']
2019-07-04 2019-10-04
22.688172043010752


In [10]:
# Fill in trend feature from row m+1 to row n-1 (starting from 0)
# Save as pkl file

def trend_df(google_trend_df, song_dropna, m, n):

    i=0
    for song in range(m, n, 1):
        
        try: 
            google_trend_df = google_trend_df.append({'artist': song_dropna.iloc[song]['artist'], 'title': song_dropna.iloc[song]['title'], 'spotify_id':song_dropna.iloc[song]['spotify_id'],'google_trend': google_trend(song_dropna, song, 3)} , ignore_index=True)
            i=i+1
        except requests.exceptions.Timeout:
            print("Timeout occurred")
        time.sleep(randint(3,10))

        #print the result to monitor the process
        print("song_df row No.", end =" ")
        print(song)
        print("Mean trend: ", end =" ")
        print(google_trend_df.iloc[i]["google_trend"])
    
    #save as pkl file
    with open('google_trend_df'+'_'+str(m)+'_'+str(n)+'.pkl', 'wb') as f:
        pickle.dump(google_trend_df, f)

In [ ]:
# To get data follow the stpes below, note that Google only allows 1,400 rows per day
# Stpe 1 change the picklefile to read at the beginning of this program
# Stpe 2 Change the parameters in the fuction below
# Stpe 3 Click run
# Step 4 (optional) check the file as shown above

# Rows after 8000
data = {'google_trend':['']} 
google_trend_df = pd.DataFrame(data)
trend_df(google_trend_df, song_dropna, 8000, len(song_dropna))

In [ ]:
# Rows from 7000 to 8000
data = {'google_trend':['']} 
google_trend_df = pd.DataFrame(data)
trend_df(google_trend_df, song_dropna, 7000, 8000)

In [ ]:
# Rows from 6000 to 7000
data = {'google_trend':['']} 
google_trend_df = pd.DataFrame(data)
trend_df(google_trend_df, song_dropna, 6000, 7000)

In [ ]:
# Rows from 5000 to 6000
data = {'google_trend':['']} 
google_trend_df = pd.DataFrame(data)
trend_df(google_trend_df, song_dropna, 5000, 6000)

In [ ]:
# Rows from 4000 to 5000
data = {'google_trend':['']} 
google_trend_df = pd.DataFrame(data)
trend_df(google_trend_df, song_dropna, 4000, 5000)

In [ ]:
# Rows from 3000 to 4000
data = {'google_trend':['']} 
google_trend_df = pd.DataFrame(data)
trend_df(google_trend_df, song_dropna, 3000, 4000)

In [ ]:
# Rows from 2000 to 3000
data = {'google_trend':['']} 
google_trend_df = pd.DataFrame(data)
trend_df(google_trend_df, song_dropna, 2000, 3000)

In [ ]:
# Rows from 1000 to 2000
data = {'google_trend':['']} 
google_trend_df = pd.DataFrame(data)
trend_df(google_trend_df, song_dropna, 1000, 2000)

In [15]:
# Rows from 0 to 1000
data = {'google_trend':['']} 
google_trend_df = pd.DataFrame(data)
trend_df(google_trend_df, song_dropna, 0, 1000)

2019-05-17
2019-05-17 00:00:00
['Lewis Capaldi']
2019-02-17 2019-05-17
song_df row No. 0
Mean trend:  34.98888888888889
2019-09-06
2019-09-06 00:00:00
['Post Malone']
2019-06-06 2019-09-06
song_df row No. 1
Mean trend:  25.172043010752688
2019-06-21
2019-06-21 00:00:00
['Shawn Mendes & Camila Cabello']
2019-03-21 2019-06-21
song_df row No. 2
Mean trend:  1.8387096774193548
2019-05-03
2019-05-03 00:00:00
['Lizzo']
2019-02-03 2019-05-03
song_df row No. 3
Mean trend:  23.833333333333332
2019-10-23
2019-10-23 00:00:00
['Selena Gomez']
2019-07-23 2019-10-23
song_df row No. 4
Mean trend:  16.387096774193548
2016-03-09
2016-03-09 00:00:00
['Lizzo']
2015-12-09 2016-03-09
song_df row No. 5
Mean trend:  14.119565217391305
2019-06-28
2019-06-28 00:00:00
['Chris Brown Featuring Drake']
2019-03-28 2019-06-28
song_df row No. 6
Mean trend:  6.193548387096774
2019-06-21
2019-06-21 00:00:00
['Lil Nas X']
2019-03-21 2019-06-21
song_df row No. 7
Mean trend:  31.236559139784948
2019-09-20
2019-09-20 00:00